<a href="https://colab.research.google.com/github/alex-smith-uwec/Placement/blob/main/SimpleMathH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start up functions to calculate MATHH read in from h.py

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
#@title Initialize functions
##Define a function F_mpt to return math placement level between 1 and 7
def F_mpt(mfund, aalg, tag):
    if np.isnan(mfund):
        return np.nan
    if mfund < 355:
        return 1
    elif mfund < 465 and aalg < 455:
        return 2
    elif mfund < 465:
        return 3
    elif aalg < 400:
        return 4
    elif aalg < 555:
        return 5
    elif aalg < 575:
        return 6
    elif aalg>575 and tag<555:
       return 6
    else:
        return 7

##varible order:
###### [gpa, mgpa, mact, mpt, sen]

## gpa is overall gradepoint average (capped at 4)
##mgpa is math gpa calculated during transcript review
##mact is math ACT
##mpt is math placement level as determined by mfund, aalg, tag
##sen is assessment of senior year math achievement. Ranges from 0 to 10. Determined during transcript review. SEE RUBRIC!!



##H_none: determines MATHH with no ACT, no MPT. Uses mgpa and sen from transcript review
def H_none(gpa, mgpa, sen):
    agpa = (gpa + mgpa) / 2

    if sen >= 8:
        if agpa >= 3.5:
            return 7
        elif agpa >= 3.25:
            return 6
        elif agpa >= 3:
            return 5
        elif agpa >= 2.5:
            return 4
        else:
            return 3
    elif sen >= 5 and sen < 8:
        if agpa >= 3.5:
            return 6
        elif agpa >= 3.25:
            return 5
        elif agpa >= 2.5:
            return 4
        else:
            return 3
    elif sen >= 3 and sen < 5:
        if agpa > 3.5:
            return 5
        elif agpa >= 3:
            return 4
        elif agpa >= 2.5:
            return 3
        else:
            return 2
    else:  # sen < 3
        if agpa > 3.5:
            return 4
        elif agpa >= 3:
            return 3
        elif agpa >= 2.5:
            return 2
        else:
            return 1

##H_mpt: determines MATHH with additional info of MPT only
def H_mpt(gpa, mgpa, mpt, sen):
    h0 = H_none(gpa, mgpa, sen)

    if mpt <= h0 - 2:
        return h0
    elif mpt<=h0:
        return h0
    else:
        return min(h0 + 1, 7)

##H_mact: determines MATHH with additional info of MACT only
def H_mact(gpa, mgpa, mact, sen):
    h0 = H_none(gpa, mgpa, sen)

    if h0 >= 4:#was h0>=5 before June 4
        return h0
    else:
        if mact >= 23:
            return 5
        elif mact == 22:
            return 4
        elif mact == 21:
            return 3
        elif mact > 16:
            return 2
        else:
            return 1


##H_both: determines MATHH with additional info of  both MACT and MPT
def H_both(gpa,mgpa,mact,mpt, sen):
    value2 = H_none(gpa, mgpa, sen)
    value3 = H_mpt(gpa, mgpa, mpt, sen)
    value6 = H_mact(gpa, mgpa, mact, sen)

    return max(value2, value3, value6)

##This function takes a dataframe with specific column names and classifies cases by "Type".
##Depending on Type, one of the four H functions will later be used to determine MATHH
def process_dataframe(df):
    df_typed = df.copy()
    df_typed['Type'] = 'x'
    ##cap GPA at 4.0
    df_typed['GPA'] = df_typed['GPA'].apply(lambda x: min(x, 4) if not pd.isna(x) else 'x')

    df_typed.loc[(df['MACT'].isna()) & (df['MGPA'].notna()) & (df['MPT'].isna()) & (df['SEN'].notna()),'Type'] = 'h_none'
    df_typed.loc[(df['MACT'].isna()) & (df['MPT'].notna())& (df['MGPA'].notna()) & (df['SEN'].notna()), 'Type'] = 'h_mpt'
    df_typed.loc[(df['MACT'].notna()) & (df['MPT'].notna()) &(df['MGPA'].isna()) & (df['SEN'].isna()), 'Type'] = 'expedited'
    df_typed.loc[(df['MACT'].notna()) & (df['MPT'].isna())& (df['MGPA'].notna()) & (df['SEN'].notna()), 'Type'] = 'h_mact'
    df_typed.loc[(df['MACT'].notna()) & (df['MPT'].notna())& (df['MGPA'].notna()) & (df['SEN'].notna()), 'Type'] = 'h_both'

    return df_typed

def apply_model(row):
    if row['Type'] == 'h_none':
        return H_none(row['GPA'], row['MGPA'], row['SEN'])
    elif row['Type'] == 'h_mpt':
        return H_mpt(row['GPA'], row['MGPA'], row['MPT'], row['SEN'])
    elif row['Type'] == 'h_mact':
        return H_mact(row['GPA'], row['MGPA'], row['MACT'], row['SEN'])
    elif row['Type'] == 'h_both':
        return H_both(row['GPA'],row['MGPA'], row['MACT'], row['MPT'], row['SEN'])
    else:
        return None



In [ ]:
#@title
#useful widget to remember variable names and function name conventions
Setup = [
    ['H_none', 1, 1, 0, 0, 1,'(gpa,mgpa,sen)'],
    ['H_mpt', 1, 1, 0, 1, 1,'(gpa,mgpa,mpt,sen)'],
    ['H_mact', 1, 1, 1, 0, 1,'(gpa,mgpa,mact,sen)'],
    ['H_both', 1, 1, 1, 1, 1,'(gpa,mgpa,mact,mpt,sen)']
]
columns = ['', 'gpa', 'mgpa', 'mact', 'mpt', 'sen','calling sequence']
Setup = pd.DataFrame(Setup, columns=columns)
widgit_title = "Widget to remember name conventions\nH is for 'Holistic'"
print(widgit_title)
print(Setup.to_string(index=False))
print('\nF_mpt(mfund,aalg,tag) for placement level as determined by placement scores')

Widget to remember name conventions
H is for 'Holistic'
        gpa  mgpa  mact  mpt  sen        calling sequence
H_none    1     1     0    0    1          (gpa,mgpa,sen)
 H_mpt    1     1     0    1    1      (gpa,mgpa,mpt,sen)
H_mact    1     1     1    0    1     (gpa,mgpa,mact,sen)
H_both    1     1     1    1    1 (gpa,mgpa,mact,mpt,sen)

F_mpt(mfund,aalg,tag) for placement level as determined by placement scores


#Examples

In [ ]:
##Example
print("Placement test level for mfund=470,aalg=300,tag=350 is F_mpt(470,300,350):")
F_mpt(470,300,35)

Placement test level for mfund=470,aalg=300,tag=350 is F_mpt(470,300,350):


4

In [ ]:
##Example
print("Scenario: placement scores indicate level 4 with gpa=2.6,mgpa=2.5,senior math=7 then MATHH is:")
H_mpt(2.6,2.5,4,7)

Scenario: placement scores indicate level 4 with gpa=2.6,mgpa=2.5,senior math=7 then MATHH is:


4

# Workspace

In [ ]:
H_none(3.3,3.5,7)

5

In [ ]:
F_mpt(380, 380, 340)

2

In [ ]:
H_mpt(3.09,2.6,2,7)

4

In [ ]:
H_both(3.56,2.22,17,7,2)

3

# Senior level notes

[Most updated data in first tab of placement spreadsheet](https://universityofwieauclaire-my.sharepoint.com/:x:/g/personal/brisbia_uwec_edu/EZ3WpzF_2WJMg_gwp4TDMRoBj4MpcfksUS6i4syzA_T1bA?e=Qdkd0C)

In [ ]:
#@title
##Edit table or recover table in case it gets deleted. string--> dictionary-->dataframe
df_dict_string='{"SEN": {"0": "9-10", "1": "8-9", "2": "7-8", "3": "6-7", "4": "5-6", "5": "4-5", "6": ".", "7": "2-3", "8": "1-2", "9": "0-1"}, "Course": {"0": "AP Calc BC", "1": "AP Calc AB", "2": "PreCalc", "3": "FST (Functions/Stat/Trig)", "4": "Algebra 3/Advanced Alg", "5": "AP Stats", "6": ".", "7": "Alg 2 or non-AP Stats", "8": "Quantitiative Reasoning", "9": "No info"}, "Notes": {"0": "grade A: 10, grade B: 9.5, lower than B or unknown: 9..similar striation for lower tiers", "1": ".", "2": "(High School Calc not AP, etc)", "3": " (Business Calc, Applied Calc, etc.)", "4": "(College Algebra, Trig, PreCalc Algebra, etc.)", "5": "(College Prep Math, Math Applications, etc.)", "6": ".", "7": "(Prob and Stats, CIS Stats, etc.)", "8": "(Liberal Arts Math, Modeling with Math, Math Problem Solving, etc.)", "9": "(Tech College Math, Personal Finance, etc.)"}}'
# Convert the string back to a dictionary
df_dict = json.loads(df_dict_string)
# Convert the dictionary to a DataFrame
df = pd.DataFrame(df_dict)
df.set_index('SEN')

,Course,Notes
SEN,,
9-10,AP Calc BC,"grade A: 10, grade B: 9.5, lower than B or unk..."
8-9,AP Calc AB,.
7-8,PreCalc,"(High School Calc not AP, etc)"
6-7,FST (Functions/Stat/Trig),"(Business Calc, Applied Calc, etc.)"
5-6,Algebra 3/Advanced Alg,"(College Algebra, Trig, PreCalc Algebra, etc.)"
4-5,AP Stats,"(College Prep Math, Math Applications, etc.)"
.,.,.
2-3,Alg 2 or non-AP Stats,"(Prob and Stats, CIS Stats, etc.)"
1-2,Quantitiative Reasoning,"(Liberal Arts Math, Modeling with Math, Math P..."
